# 1. Extracting MFCC features from an audio signal, step by step

Imports

In [ ]:
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
import os
import scipy
import scipy.fftpack as fft
from scipy.io import wavfile
from scipy.signal import get_window

%matplotlib inline

### Play your .wav file

You can record your own .wav audio online - https://voice-recorder-online.com/.

In [ ]:
audio_file = "audios/example3.wav"  # replace the filepath accordingly
ipd.Audio(audio_file)

### Read the audio signal into an array

In [ ]:
sample_rate, audio = wavfile.read(audio_file)

# Convert to mono channel if stereo
if len(audio.shape) == 2:
    audio = audio.sum(axis=1) / 2

# Normalize the audio signal
def normalize_audio(audio):
    audio = audio / np.max(np.abs(audio))
    return audio

audio = normalize_audio(audio)
    
print(f"Sample rate: {sample_rate}Hz")
print(f"Length: {round(len(audio) / sample_rate, 4)}s")
print(f"Audio signal shape: {audio.shape}")

### Plot the waveform

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(np.linspace(0, len(audio) / sample_rate, num=len(audio)), audio)
plt.title("Waveform")
plt.grid(True)

### Extract time frames

In [ ]:
def frame_audio(audio, frame_n_samples, shift_n_samples):
    audio = np.pad(audio, int(frame_n_samples / 2), mode='reflect')
    frame_num = int((len(audio) - frame_n_samples) / shift_n_samples) + 1
    frames = np.zeros((frame_num, frame_n_samples))
    
    for n in range(frame_num):
        frames[n] = audio[n * shift_n_samples: n * shift_n_samples + frame_n_samples]
    
    return frames

shift_size = 10  # ms
frame_size = 25  # ms

frame_n_samples = np.round(sample_rate * frame_size / 1000).astype(int)
shift_n_samples = np.round(sample_rate * shift_size / 1000).astype(int)

audio_framed = frame_audio(audio, frame_n_samples, shift_n_samples)
print(f"Framed audio shape: {audio_framed.shape}")
print(f"Frame - number of samples: {frame_n_samples}")
print(f"Shift - number of samples: {shift_n_samples}")

### Plot overlapping time frames

In [ ]:
index = 32
fig, axes = plt.subplots(2, 2, figsize=(25, 10))
plt.axes(axes[0, 0])
axes[0, 0].set_ylim(-0.1, 0.1)
plt.plot(audio_framed[index])
plt.title(f"{index}th time frame")
plt.grid(True)
index += 1
plt.axes(axes[0, 1])
axes[0, 1].set_ylim(-0.1, 0.1)
plt.plot(audio_framed[index])
plt.title(f"{index}th time frame")
plt.grid(True)
index += 1
plt.axes(axes[1, 0])
axes[1, 0].set_ylim(-0.1, 0.1)
plt.plot(audio_framed[index])
plt.title(f"{index}th time frame")
plt.grid(True)
index += 1
plt.axes(axes[1, 1])
axes[1, 1].set_ylim(-0.1, 0.1)
plt.plot(audio_framed[index])
plt.title(f"{index}th time frame")
plt.grid(True)

plt.show()

### Convert to frequency domain and compute power spectrum

In [ ]:
window = get_window("hann", frame_n_samples, fftbins=True)
audio_win = audio_framed * window

audio_winT = np.transpose(audio_win)

audio_fft = np.empty((int(1 + frame_n_samples // 2), audio_winT.shape[1]), dtype=np.complex64, order='F')

for n in range(audio_fft.shape[1]):
    audio_fft[:, n] = fft.fft(audio_winT[:, n], axis=0)[:audio_fft.shape[0]]

audio_fft = np.transpose(audio_fft)

audio_power = np.square(np.abs(audio_fft))

plt.figure(figsize=(15,6))
plt.plot(audio_power[index])
plt.title(f"{index}th time frame's power spectrum")
plt.grid(True)

### Compute Mel filter banks

In [ ]:
freq_min = 0
freq_high = sample_rate / 2
mel_filter_num = 10

print(f"Minimum frequency in the audio signal: {freq_min}")
print(f"Maximum frequency in the audio signal: {freq_high}")
print()

def freq_to_mel(freq):
    return 2595.0 * np.log10(1.0 + freq / 700.0)

def met_to_freq(mels):
    return 700.0 * (10.0**(mels / 2595.0) - 1.0)

def get_filter_points(fmin, fmax, mel_filter_num, fft_size, sample_rate=44100):
    fmin_mel = freq_to_mel(fmin)
    fmax_mel = freq_to_mel(fmax)
    
    print("MEL min: {0}".format(fmin_mel))
    print("MEL max: {0}".format(fmax_mel))
    
    mels = np.linspace(fmin_mel, fmax_mel, num=mel_filter_num+2)
    freqs = met_to_freq(mels)
    
    return np.floor((fft_size + 1) / sample_rate * freqs).astype(int), freqs

filter_points, mel_freqs = get_filter_points(freq_min, freq_high, mel_filter_num, frame_n_samples, sample_rate=44100)
print()
print("Mel filter points:")
filter_points

In [ ]:
def get_filters(filter_points, fft_size):
    filters = np.zeros((len(filter_points)-2,int(fft_size/2+1)))
    
    for n in range(len(filter_points)-2):
        filters[n, filter_points[n] : filter_points[n + 1]] = np.linspace(0, 1, filter_points[n + 1] - filter_points[n])
        filters[n, filter_points[n + 1] : filter_points[n + 2]] = np.linspace(1, 0, filter_points[n + 2] - filter_points[n + 1])
    
    return filters

filters = get_filters(filter_points, frame_n_samples)

enorm = 2.0 / (mel_freqs[2:mel_filter_num+2] - mel_freqs[:mel_filter_num])
filters *= enorm[:, np.newaxis]

plt.figure(figsize=(15,4))
plt.title("Mel filters")
for n in range(filters.shape[0]):
    plt.plot(filters[n])

### Apply Mel filters

In [ ]:
audio_filtered = np.dot(filters, np.transpose(audio_power))
audio_log = 10.0 * np.log10(audio_filtered)
print(f"Filtered signal shape: {audio_log.shape}")

plt.figure(figsize=(15,4))
plt.title("Filtered signal")
plt.imshow(audio_log, origin="lower", cmap="gray")

### Extract MFCC features

In [ ]:
def dct(dct_filter_num, filter_len):
    basis = np.empty((dct_filter_num,filter_len))
    basis[0, :] = 1.0 / np.sqrt(filter_len)
    
    samples = np.arange(1, 2 * filter_len, 2) * np.pi / (2.0 * filter_len)

    for i in range(1, dct_filter_num):
        basis[i, :] = np.cos(i * samples) * np.sqrt(2.0 / filter_len)
        
    return basis

dct_filter_num = 40
dct_filters = dct(dct_filter_num, mel_filter_num)
# Extract the first 13 coefficients
cepstral_coefficents = np.dot(dct_filters, audio_log)[:13,:]

print(f"MFCC features shape: {cepstral_coefficents.shape}")

In [ ]:
plt.figure(figsize=(15,5))
plt.title("MFCC features")
plt.imshow(cepstral_coefficents, cmap="gray", origin="lower");